In [2]:
import pandas as pd
import numpy as np
import my_def

<h1>데이터확인</h1>

In [3]:
df = pd.read_csv('Merged_Sales_data_v2.6(selected_store).csv') # 월매출액 / 점포수

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335256 entries, 0 to 335255
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   년분기        335256 non-null  int64 
 1   행정동        335256 non-null  object
 2   업종명        335256 non-null  object
 3   월매출(점포)    335256 non-null  int64 
 4   인구수        335256 non-null  int64 
 5   지역생활인구     335256 non-null  int64 
 6   장기외국인      335256 non-null  int64 
 7   단기외국인      335256 non-null  int64 
 8   주차장면적(면)   335256 non-null  int64 
 9   주차장개수(개소)  335256 non-null  int64 
 10  학교수        335256 non-null  int64 
 11  학생수        335256 non-null  int64 
 12  버스정류장수     335256 non-null  int64 
dtypes: int64(11), object(2)
memory usage: 33.3+ MB


In [5]:
df = my_def.reduce_memory_usage(df) # 메모리 사용량 감소를 위한 Dtype 변환

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335256 entries, 0 to 335255
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   년분기        335256 non-null  int16 
 1   행정동        335256 non-null  object
 2   업종명        335256 non-null  object
 3   월매출(점포)    335256 non-null  int64 
 4   인구수        335256 non-null  int32 
 5   지역생활인구     335256 non-null  int32 
 6   장기외국인      335256 non-null  int32 
 7   단기외국인      335256 non-null  int32 
 8   주차장면적(면)   335256 non-null  int32 
 9   주차장개수(개소)  335256 non-null  int16 
 10  학교수        335256 non-null  int16 
 11  학생수        335256 non-null  int16 
 12  버스정류장수     335256 non-null  int16 
dtypes: int16(5), int32(5), int64(1), object(2)
memory usage: 17.3+ MB
None


<h1>전체업종 선형회귀 모델</h1>
Polynomial Features & Standard Scaler 사용 안함

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# '행정동' 컬럼 원핫인코딩 수행
df = pd.get_dummies(df, columns=['행정동'], drop_first=True)

# 피처와 타겟 설정
features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df.columns if col.startswith('행정동_')]
target = '월매출(점포)'

X = df[features]
y = df[target]

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 선형 회귀 모델 학습 및 예측
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 모델 평가 호출 선형회귀모델
my_def.evaluate_model(model, X_train, X_test, y_train, y_test)

{'train_mse': np.float64(8308676338803392.0),
 'test_mse': np.float64(1.007356115630108e+16),
 'train_rmse': np.float64(91151940.95),
 'test_rmse': np.float64(100367131.85),
 'train_r2': 0.05,
 'test_r2': 0.06,
 'train_mae': np.float64(45549975.43),
 'test_mae': np.float64(46656048.24),
 'train_mae_ratio': np.float64(99.37),
 'test_mae_ratio': np.float64(98.81)}

<h1>개별업종 선형모델 평가</h1>
Polynomial Features & Standard Scaler 사용 안함

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.6(selected_store).csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)


                        train_mse                       test_mse   train_rmse  \
미곡판매          5933382120252120.00 187486351853406446541602816.00  77028450.07   
분식전문점           66117949502714.05              50457373774981.29   8131294.45   
편의점            565354458427281.50             671657762850320.62  23777183.57   
한식음식점           83970274937360.14             119656390845344.66   9163529.61   
의약품           3297095806791203.00            4385685765224285.00  57420343.14   
슈퍼마켓           774889047348111.38             911779768361101.50  27836828.97   
반찬가게          1647622160283863.00            7353558927026090.00  40590912.29   
호프-간이주점        199863041084321.25             234378647239989.62  14137292.57   
커피-음료           43584768782683.19              49221538455665.95   6601876.16   
일반의원          1139955314654611.50            1331702679340440.25  33763224.29   
치과의원           816254335972981.50            1209278175144936.75  28570165.14   
육류판매          21479329285198

In [11]:
print(results_df.loc["한식음식점"])

train_mse              83970274937360.14
test_mse              119656390845344.66
train_rmse                    9163529.61
test_rmse                    10938756.37
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max            1517325.70
test_mae_ratio_min                 15.42
test_mae_ratio_mean             27126.84
Name: 한식음식점, dtype: float64


In [12]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
미곡판매         1517325.70
핸드폰               61.39
전자상거래업            60.52
부동산중개업            53.16
인테리어              52.72
시계및귀금속            50.45
철물점               48.60
컴퓨터및주변장치판매        46.46
가전제품수리            45.02
완구                44.49
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64


<h1>Standard Scaler 적용</h1>

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.6(selected_store).csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # Standard Scaler 적용
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)

                        train_mse  \
미곡판매          5933382120251782.00   
분식전문점           66117949502714.11   
편의점            565354458427282.12   
한식음식점           83970274937359.55   
의약품           3297095806791198.50   
슈퍼마켓           774889047348110.50   
반찬가게          1647662437419459.25   
호프-간이주점        199863041084321.91   
커피-음료           43584768782683.46   
일반의원          1139955314654609.75   
치과의원           816254335972981.50   
육류판매          2147932928519835.50   
자동차수리          251019654067631.84   
한의원            242613758118259.75   
의료기기         11004942879723562.00   
양식음식점          171178063440133.22   
세탁소             13313948790821.29   
피부관리실           49764153953883.31   
화장품            429781916304557.75   
철물점           1462659109904726.00   
스포츠 강습          70783549736711.34   
스포츠클럽         1355505789017290.00   
골프연습장          698601328617442.25   
미용실             12480777376634.33   
신발            1908400634568763.25   
안경             170357175529887.72   
일

In [14]:
print(results_df.loc["한식음식점"])

train_mse              83970274937359.55
test_mse              119656391108593.31
train_rmse                    9163529.61
test_rmse                    10938756.38
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max    252965611972586.56
test_mae_ratio_min                 15.42
test_mae_ratio_mean    10094752810748.29
Name: 한식음식점, dtype: float64


In [15]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
시계및귀금속         252965611972586.56
자전거 및 기타운송장비   106151680026948.19
가방              66407262943700.05
전자상거래업          49897389501089.62
인테리어            36197484551505.90
컴퓨터및주변장치판매      28534615058272.08
가전제품수리          20195298939560.32
반찬가게             4708890177090.14
자동차미용             242893750677.25
섬유제품                5030478837.16
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64


<h1>Min-Max Scaler 적용</h1>

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 파일 로드
file_path = 'Merged_Sales_data_v2.6(selected_store).csv'
df = pd.read_csv(file_path)

# 업종명 고유값 추출
unique_categories = df["업종명"].unique()

# 결과를 저장할 딕셔너리 생성
results = {}

# 각 업종별로 데이터를 나누어 모델 학습 및 예측 수행
for category in unique_categories:
    # 업종별 데이터 필터링
    df_category = df[df['업종명'] == category]
    
    # 원핫인코딩 수행
    df_category = pd.get_dummies(df_category, columns=['행정동'], drop_first=True)
    
    # 피처와 타겟 설정
    features = ['년분기', '인구수', '지역생활인구', '장기외국인', '단기외국인', '주차장면적(면)', '주차장개수(개소)', '학교수', '학생수', '버스정류장수'] + [col for col in df_category.columns if col.startswith('행정동_')]
    target = '월매출(점포)'
    
    # 피처와 타겟 데이터 분리
    X = df_category[features]
    y = df_category[target]
    
    # Min-Max Scaler 적용
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 학습 데이터와 테스트 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 성능 평가 호출
    evaluation_result = my_def.evaluate_model(model, X_train, X_test, y_train, y_test)
    
    # 결과 저장
    if evaluation_result is not None:
        results[category] = evaluation_result

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.apply(pd.to_numeric)

# test_mae_ratio의 최고, 최저, 평균 추가
test_mae_ratio_max = results_df['test_mae_ratio'].max()
test_mae_ratio_min = results_df['test_mae_ratio'].min()
test_mae_ratio_mean = results_df['test_mae_ratio'].mean()

# 모든 행에 공통으로 최고, 최저, 평균 추가
test_mae_summary = {
    'test_mae_ratio_max': test_mae_ratio_max,
    'test_mae_ratio_min': test_mae_ratio_min,
    'test_mae_ratio_mean': test_mae_ratio_mean
}

for key, value in test_mae_summary.items():
    results_df[key] = value

# 결과를 소수점 2자리로 포맷팅하여 출력
pd.options.display.float_format = '{:.2f}'.format
print(results_df)

                        train_mse  \
미곡판매          5933382120251781.00   
분식전문점           66117949502714.12   
편의점            565354458427281.88   
한식음식점           83970274937359.58   
의약품           3297095806791197.50   
슈퍼마켓           774889047348110.50   
반찬가게          1647622267706401.00   
호프-간이주점        199863041084322.72   
커피-음료           43584768782683.46   
일반의원          1139955314654608.50   
치과의원           816254335972981.50   
육류판매          2147932928519835.00   
자동차수리          251019654067631.75   
한의원            242613758118259.75   
의료기기         11004942879723568.00   
양식음식점          171178063440133.28   
세탁소             13313948790821.30   
피부관리실           49764153953883.30   
화장품            429781916304557.81   
철물점           1462659109904726.00   
스포츠 강습          70783549736711.31   
스포츠클럽         1355508443981607.50   
골프연습장          698601396944354.50   
미용실             12480777376634.32   
신발            1908400634568763.00   
안경             170357175529887.69   
일

In [17]:
print(results_df.loc["한식음식점"])

train_mse              83970274937359.58
test_mse              119656391108595.05
train_rmse                    9163529.61
test_rmse                    10938756.38
train_r2                            0.84
test_r2                             0.80
train_mae                     6645764.80
test_mae                      7584013.79
train_mae_ratio                    15.59
test_mae_ratio                     16.87
test_mae_ratio_max     32981049624083.46
test_mae_ratio_min                 15.42
test_mae_ratio_mean      827568842930.97
Name: 한식음식점, dtype: float64


In [18]:
# test_mae_ratio 값이 큰 순으로 상위 10개 추출
top_10_test_mae = results_df['test_mae_ratio'].nlargest(10)

# test_mae_ratio 값이 작은 순으로 하위 10개 추출
tail_10_test_mae = results_df['test_mae_ratio'].nsmallest(10)

# 결과 출력
print("Top 10 test_mae_ratio:")
print(top_10_test_mae)
print("*" * 50)
print("Bottom 10 test_mae_ratio:")
print(tail_10_test_mae)

Top 10 test_mae_ratio:
가방             32981049624083.46
시계및귀금속          4459450181306.03
인테리어            2343634202026.49
스포츠클럽           1731362971174.72
자전거 및 기타운송장비    1145772796062.68
섬유제품             996814506488.66
컴퓨터및주변장치판매       753488390826.30
전자상거래업           737366711614.68
가전제품수리           404826123455.85
반찬가게             292374927770.65
Name: test_mae_ratio, dtype: float64
**************************************************
Bottom 10 test_mae_ratio:
편의점     15.42
미용실     15.97
한식음식점   16.87
치과의원    17.54
안경      17.75
일반의원    18.15
의약품     18.25
자동차수리   19.06
한의원     19.19
슈퍼마켓    20.09
Name: test_mae_ratio, dtype: float64
